<h2><strong>Hollywood Film Locations Web Scrape</strong></h2>

<p>Scrape a film's shooting locations from the Internet Movie Database (www.imdb.com).</p><p>A given film's shooting locations are listed on its own page (www.imdb.com/title/unique_film_id/locations?ref_=tt_dt_loc).</p>  <p>A link to this page can be found under the Details subsection on the film's main page.</p><p>On the locations page, each filming location is listed as link. Some of the listings include the scene which was filmed at the given location</p><p>This project grabs all the text from IMDB and places it in an Microsoft Excel sheet for further cleaning and exam.</p>

In [ ]:
 # customary imports for getting web page and scraping it
import requests
from bs4 import BeautifulSoup

In [ ]:
# Get website url, and parse it
# This example requests from the Internet Movie Database the filming locations web page for the 1978 Horror film Halloween
page = requests.get(
    "https://www.imdb.com/title/tt0077651/locations?ref_=tt_dt_loc")

soup = BeautifulSoup(page.content, 'html.parser')


In [ ]:
# Preview the metadata for the film's URL id
meta = soup.select("head meta")
print(meta)

In [ ]:
# Get the film's unique ID for potential dynamic URL function argument (input)
id = soup.find("meta", property="pageId")
print(id["content"] if id else "None found")


In [ ]:
# Get the web page's filming locations section with id=filming_locations
locations = soup.find(id="filming_locations")
print(locations)

In [ ]:
# Get the locations, each one listed within the soda class
locs = locations.findAll(class_="soda")
print(locs)

In [ ]:
# Get the anchor tags which include the individual location text
places = locations.select("dt a")
print(places)

In [ ]:
# Test and confirm that I can get the text I want
one = places[0].get_text()
print(one)

In [ ]:
# Create a list of all the location strings
addresses = [p.get_text() for p in places]
print(addresses)

In [ ]:
# Iterate through list of strings, remove newline character
for i, e in enumerate(addresses):
    addresses[i] = e.replace('\n', '')
    print(addresses)

In [ ]:
# Get scene descriptions (if any) for each location
scene = locations.select("dd")
print(scene)

In [ ]:
# Iterate through list of scenes, remove newline character
scenes = [s.get_text() for s in scene]
for i, e in enumerate(scenes):
    scenes[i] = e.replace('\n', '')
print(scenes)

In [ ]:
# Iterate through list of scenes, remove parenthesis characters
for i, e in enumerate(scenes):
    scenes[i] = e.replace('(', '').replace(')', '')
print(scenes)

<h2><strong>Cleaning Text</strong></h2>

In [ ]:
# import Python Pandas 
import pandas as pd

In [ ]:
# Create Pandas Dataframe of scene description and locations
film_locations = pd.DataFrame({
    "scene": scenes, 
    "address": addresses
})
film_locations

In [ ]:
# Remove all Null (blank) scene description rows
index_loc = film_locations[film_locations['scene'] == ''].index
film_locations.drop(index_loc, inplace=True)
film_locations

In [ ]:
# Get film title to use within function coded below
name = soup.select("h3 a")
title  = name[0].get_text()

<h2><strong>Put it in a function</strong></h2>

In [ ]:
# Export Pandas Dataframe to Microsoft Excel sheet
film_locations.to_excel("halloween1978_filmLocations.xlsx")

In [ ]:
# Python function to automate process
def scrapeImdbLocations(url):
   path = url
   # imports
   import requests
   from bs4 import BeautifulSoup
   import pandas as pd
   # get web site
   page = requests.get(path)
   soup = BeautifulSoup(page.content, 'html.parser')
   # get film title
   name = soup.select("h3 a")
   title = name[0].get_text()
   # get film locations
   locations = soup.find(id="filming_locations")
   places = locations.select("dt a")
   # remove newline chars from location address
   addresses = [p.get_text() for p in places]
   for i, e in enumerate(addresses):
    addresses[i] = e.replace('\n', '')
   # remove newline and parenthesis from location scene info
   scene = locations.select("dd")
   scenes = [s.get_text() for s in scene]
   for i, e in enumerate(scenes):
    scenes[i] = e.replace('\n', '').replace('(', '').replace(')', '')
   # create Pandas dataframe
   film_locations = pd.DataFrame({
     "scene": scenes, 
     "address": addresses
   })
   # remove empty locations
   index_loc = film_locations[film_locations['scene'] == ''].index
   film_locations.drop(index_loc, inplace=True)
   # export dataframe to MS Excel sheet
   film_locations.to_excel(title+"_filmLocations.xlsx")

    

    
   
   
   
   



In [ ]:
# Test above function, supplying it with the 1991 film Dances with Wolves film locations URL
scrapeImdbLocations(
    'https://www.imdb.com/title/tt0099348/locations?ref_=tt_dt_loc')
